# 快速入门

在开始之前确保安装所有必要的库

In [9]:
!pip install transformers datasets evaluate accelerate

## pipeline

### 基础调用

任务	      描述	                  模态	     Pipeline

文本分类	为给定的文本序列分配一个标签	NLP	pipeline(task=“sentiment-analysis”)

文本生成	根据给定的提示生成文本	NLP	pipeline(task=“text-generation”)

命名实体识别	为序列里的每个 token 分配一个标签（人, 组织, 地址等等）	NLP	pipeline(task=“ner”)

问答系统	通过给定的上下文和问题, 在文本中提取答案	NLP	pipeline(task=“question-answering”)

掩盖填充	预测出正确的在序列中被掩盖的token	NLP	pipeline(task=“fill-mask”)

文本摘要	为文本序列或文档生成总结	NLP	pipeline(task=“summarization”)

文本翻译	将文本从一种语言翻译为另一种语言	NLP	pipeline(task=“translation”)

图像分类	为图像分配一个标签	Computer vision	pipeline(task=“image-classification”)

图像分割	为图像中每个独立的像素分配标签（支持语义、全景和实例分割）	Computer vision	pipeline(task=“image-segmentation”)

目标检测	预测图像中目标对象的边界框和类别	Computer vision	pipeline(task=“object-detection”)

音频分类	给音频文件分配一个标签	Audio	pipeline(task=“audio-classification”)

自动语音识别	将音频文件中的语音提取为文本	Audio	pipeline(task=“automatic-speech-recognition”)

视觉问答	给定一个图像和一个问题，正确地回答有关图像的问题	Multimodal	pipeline(task=“vqa”)

创建一个 pipeline() 实例并且指定你想要将它用于的任务与要使用的模型。若不指定模型pipeline会使用该任务默认的模型

In [10]:
import torch
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("We are very happy to show you the 🤗 Transformers library.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

In [18]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
print(results)
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

[{'label': 'POSITIVE', 'score': 0.9997795224189758}, {'label': 'NEGATIVE', 'score': 0.5308618545532227}]
label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


pipeline() 也可以为任何你喜欢的任务遍历整个数据集。在下面这个示例中，让我们选择自动语音识别作为我们的任务：

In [11]:
speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

In [19]:
from datasets import load_dataset, Audio

dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")
result = speech_recognizer(dataset[:4]["audio"])#将数据集中的采样频率和模型训练数据保持一致
print([d["text"] for d in result])

The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split: 0 examples [00:00, ? examples/s]

['I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT', "ONDERING HOW I'D SET UP A JOIN TO HELL TWITH MY WIFE AND WHERE THE AP MIGHT BE", "I I'D LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER I'M NOT SEEING THE OPTION TO DO IT ON THE APSO I CALLED IN TO GET SOME HELP CAN I JUST DO IT OVER THE PHONE WITH YOU AND GIVE YOU THE INFORMATION OR SHOULD I DO IT IN THE AP HAN I'M MISSING SOMETHING UQUET HAD PREFERRED TO JUST DO IT OVER THE PHONE OF POSSIBLE THINGS", 'HOW DO I FURN A JOIN A COUNT']


In [22]:
dataset.shape

(563, 6)

In [24]:
for data in dataset:
    print(data)
    break

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/en-US~JOINT_ACCOUNT/602ba55abb1e6d0fbce92065.wav', 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/en-US~JOINT_ACCOUNT/602ba55abb1e6d0fbce92065.wav', 'array': array([ 0.        ,  0.00024414, -0.00024414, ..., -0.00024414,
        0.        ,  0.        ]), 'sampling_rate': 8000}, 'transcription': 'I would like to set up a joint account with my partner', 'english_transcription': 'I would like to set up a joint account with my partner', 'intent_class': 11, 'lang_id': 4}


In [31]:
from datasets import load_dataset
import soundfile as sf
import IPython.display as ipd

# 播放第一条语音数据
audio_array = dataset[2]["audio"]["array"]
sampling_rate = dataset[2]["audio"]["sampling_rate"]
sf.write('temp.wav', audio_array, sampling_rate)  # 保存为临时文件
ipd.Audio('temp.wav')

pipeline完整任务列表：https://huggingface.co/docs/transformers/v4.40.2/zh/main_classes/pipelines#pipelines

In [28]:
print(result)

[{'text': 'I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT'}, {'text': "ONDERING HOW I'D SET UP A JOIN TO HELL TWITH MY WIFE AND WHERE THE AP MIGHT BE"}, {'text': "I I'D LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER I'M NOT SEEING THE OPTION TO DO IT ON THE APSO I CALLED IN TO GET SOME HELP CAN I JUST DO IT OVER THE PHONE WITH YOU AND GIVE YOU THE INFORMATION OR SHOULD I DO IT IN THE AP HAN I'M MISSING SOMETHING UQUET HAD PREFERRED TO JUST DO IT OVER THE PHONE OF POSSIBLE THINGS"}, {'text': 'HOW DO I FURN A JOIN A COUNT'}]
